# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

 # План работы

1) Загрузим и подготовим данные

2) Обучим и проверим модель для каждого региона:
* Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.
* Обучим модель и сделаем предсказания на валидационной выборке.
* Сохраним предсказания и правильные ответы на валидационной выборке.
* Напечатаем на экране средний запас предсказанного сырья и RMSE модели.
* Проанализируем результаты.

3) Подготовим к расчёту прибыли:
* Все ключевые значения для расчётов сохраним в отдельных переменных.
* Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе. 
* Напишем выводы по этапу подготовки расчёта прибыли.

3.1) Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
* Выберем скважины с максимальными значениями предсказаний. 
* Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
* Рассчитаем прибыль для полученного объёма сырья.

5) Посчитаем риски и прибыль для каждого региона:
* Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
* Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
* Напишем выводы: предложите регион для разработки скважин и обоснуйте выбор.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from numpy.random import RandomState
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle


In [2]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
df1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
df3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


По данным видно что они готовы к исследованию. По скольку по условию задачи нам нужно использовать линейную регрессию, масштабировать данные не нужно

## Обучение и проверка модели

### Обучим и проверим df1

#### Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.

In [6]:
rs = 42
ts = 0.25

In [7]:
df1_train, df1_valid = train_test_split(df1, test_size=ts, random_state=rs)

df1_features_train = df1_train.drop(['id', 'product'], axis=1)
df1_target_train = df1_train['product']
df1_features_valid = df1_valid.drop(['id', 'product'], axis=1)
df1_target_valid = df1_valid['product']

print(df1_features_train.shape)
print(df1_target_train.shape)
print(df1_features_valid.shape)
print(df1_target_valid.shape)

(75000, 3)
(75000,)
(25000, 3)
(25000,)


In [8]:
df2_train, df2_valid = train_test_split(df2, test_size=ts, random_state=rs)

df2_features_train = df2_train.drop(['id', 'product'], axis=1)
df2_target_train = df2_train['product']
df2_features_valid = df2_valid.drop(['id', 'product'], axis=1)
df2_target_valid = df2_valid['product']

print(df2_features_train.shape)
print(df2_target_train.shape)
print(df2_features_valid.shape)
print(df2_target_valid.shape)

(75000, 3)
(75000,)
(25000, 3)
(25000,)


In [9]:
df3_train, df3_valid = train_test_split(df3, test_size=ts, random_state=rs)

df3_features_train = df3_train.drop(['id', 'product'], axis=1)
df3_target_train = df3_train['product']
df3_features_valid = df3_valid.drop(['id', 'product'], axis=1)
df3_target_valid = df3_valid['product']

print(df3_features_train.shape)
print(df3_target_train.shape)
print(df3_features_valid.shape)
print(df3_target_valid.shape)

(75000, 3)
(75000,)
(25000, 3)
(25000,)


Выборки разделены корректно

#### Обучим модель и сделаем предсказания на валидационной выборке

In [10]:
def model_linear(features_train, target_train, features_valid, target_valid):

    model = LinearRegression().fit(features_train, target_train)
    prediction = model.predict(features_valid)
    prediction_mean = prediction.mean()
    RMSE = mean_squared_error(target_valid, prediction)**0.5
    print('Cреднее значение предсказания запасов:', prediction_mean)
    print('RMSE:', RMSE)
    print('--------------------------------------------------------')

In [13]:
df1_model = LinearRegression().fit(df1_features_train, df1_target_train)
df1_prediction = df1_model.predict(df1_features_valid)
df1_prediction_mean = df1_prediction.mean()
df1_RMSE = mean_squared_error(df1_target_valid, df1_prediction)**0.5
df1_valid['prediction'] = df1_prediction

df2_model = LinearRegression().fit(df2_features_train, df2_target_train)
df2_prediction = df2_model.predict(df2_features_valid)
df2_prediction_mean = df2_prediction.mean()
df2_RMSE = mean_squared_error(df2_target_valid, df2_prediction)**0.5
df2_valid['prediction'] = df2_prediction

df3_model = LinearRegression().fit(df3_features_train, df3_target_train)
df3_prediction = df3_model.predict(df3_features_valid)
df3_prediction_mean = df3_prediction.mean()
df3_RMSE = mean_squared_error(df3_target_valid, df3_prediction)**0.5
df3_valid['prediction'] = df3_prediction

print('Первое месторождение')
print('Cреднее значение предсказания запасов:', df1_prediction_mean)
print('RMSE:', df1_RMSE)
print('---------------------------------------------------')

print('Второе месторождение')
print('Cреднее значение предсказания запасов:', df2_prediction_mean)
print('RMSE:', df2_RMSE)
print('---------------------------------------------------')

print('Третье месторождение')
print('Cреднее значение предсказания запасов:', df3_prediction_mean)
print('RMSE:', df3_RMSE)
print('---------------------------------------------------')

Первое месторождение
Cреднее значение предсказания запасов: 92.39879990657768
RMSE: 37.75660035026169
---------------------------------------------------
Второе месторождение
Cреднее значение предсказания запасов: 68.71287803913762
RMSE: 0.890280100102884
---------------------------------------------------
Третье месторождение
Cреднее значение предсказания запасов: 94.77102387765939
RMSE: 40.145872311342174
---------------------------------------------------


/tmp/ipykernel_190/2334841345.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_valid['prediction'] = df1_prediction
/tmp/ipykernel_190/2334841345.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_valid['prediction'] = df2_prediction
/tmp/ipykernel_190/2334841345.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

Вывод: Исследование показало, что наиболее больше значение предсказания запасов у месторождения 1 и 3, но покатель RMSE (средняя квадратичная ошибка), у месторождения №2 

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраним в отдельных переменных

In [14]:
budget = 10000000000/200 #Бюджет на разработку 1 из 200 скважин в регионе
income = 450000 #доход с единицы продукта
sufficient_volume = budget / income #количество тонн единиц продукта, доступное на бюджет

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [15]:
print('Достаточный объём сырья для безубыточной разработки новой скважины:', round(sufficient_volume, 2)) 

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11


Судя по показателям предсказания среднего объема скважин, особенно сомнтельным кажется месторождение df2

In [16]:
print('Cреднее значение предсказания запасов месторождения df1:', round(df1_prediction_mean, 2))
print('Cреднее значение предсказания запасов месторождения df2:', round(df2_prediction_mean, 2))
print('Среднее значение предсказания запасов месторождения df3:', round(df3_prediction_mean, 2))

Cреднее значение предсказания запасов месторождения df1: 92.4
Cреднее значение предсказания запасов месторождения df2: 68.71
Среднее значение предсказания запасов месторождения df3: 94.77


## Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [17]:
def profit_calc(prediction, target, count):
    prediction = prediction.reset_index(drop=True)
    target = target.reset_index(drop=True)
    prediction_sort = prediction.sort_values(ascending=False)
    selected = target[prediction_sort.index][:count]
    if sorted(prediction_sort[:200].index) != sorted(selected.index):
        print("INDEX ERROR")
    return (selected.sum()*income)-10000000000
    # Функция для расчёта прибыли

In [21]:
df1_profit = profit_calc(df1_valid['prediction'], df1_valid['product'], 200)
df2_profit = profit_calc(df2_valid['prediction'], df2_valid['product'], 200)
df3_profit = profit_calc(df3_valid['prediction'], df3_valid['product'], 200)
print('Прибыль с разработки 200 лучших скважин первого месторождения:', round(df1_profit), 'рублей')
print('Прибыль с разработки 200 лучших скважин второго месторождения:', round(df2_profit), 'рублей')
print('Прибыль с разработки 200 лучших скважин третьего месторождения:', round(df3_profit), 'рублей')

Прибыль с разработки 200 лучших скважин первого месторождения: 3359141114 рублей
Прибыль с разработки 200 лучших скважин второго месторождения: 2415086697 рублей
Прибыль с разработки 200 лучших скважин третьего месторождения: 2598571759 рублей


In [22]:
df1_valid['product']

75721    122.073350
80184     48.738540
19864    131.338088
76699     88.327757
92991     36.959266
            ...    
21271    148.821520
34014    123.454003
81355     91.945213
65720    149.295563
11627     57.244039
Name: product, Length: 25000, dtype: float64

Исходя из предсказаний модельной регрессии, наиболее перспективным выгядит первое месторождение

## Посчитаем риски и прибыль для каждого региона

### Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [23]:
def boots (target, prediction):
    state = np.random.RandomState(12345)
    values = []
    counter=0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = prediction[target_subsample.index]
        values.append(profit_calc(preds_subsample, target_subsample, 200))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
    print("Среднее значение", (values.mean()))
    print("Верхний квантиль", higher)
    print("Нижний квантиль", lower)
    print('Вероятность убытков',stats.percentileofscore(values, 0),'%')
    print()
print("Первое месторождение")
boots(df1_valid['product'], df1_valid['prediction'])
print()
print("Второе месторождение")
boots(df2_valid['product'], df2_valid['prediction'])
print()
print("Третье месторождение")
boots(df3_valid['product'], df3_valid['prediction'])

Первое месторождение
Среднее значение 406278783.42441905
Верхний квантиль 911737050.7514055
Нижний квантиль -117742136.49486831
Вероятность убытков 6.7 %


Второе месторождение
Среднее значение 432624131.8131374
Верхний квантиль 815972526.2857513
Нижний квантиль 16846174.932430126
Вероятность убытков 1.9000000000000001 %


Третье месторождение
Среднее значение 377362192.4229165
Верхний квантиль 901772131.3864455
Нижний квантиль -170780417.7057271
Вероятность убытков 7.4 %



# Вывод: 

1) Загрузили и подготовили данные

2) Обучили и проверили модель для каждого региона:
* Исследование показало, что наиболее больше значение предсказания запасов у месторождения 1 и 3, но покатель RMSE (средняя квадратичная ошибка), у месторождения №2

3) Подготовили к расчёту прибыли:
* Судя по показателям предсказания среднего объема скважин, особенно сомнтельным кажется месторождение df2


4) Написали функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
* Исходя из предсказаний модельной регрессии, наиболее перспективным выгядит первое месторождение

5) Посчитали риски и прибыль для каждого региона:
* Наиболее подходящее месторождение - второе, так как имеет наименьшие риски и наибольшую прибыль
